In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options

options = Options()
# options.add_argument('--headless')
driver = webdriver.Edge(options=options)

driver.get("https://skycoach.gg/")

# Click the "Choose a game" button to open the menu
# choose_game_btn = driver.find_element(By.CSS_SELECTOR, ".header__menu-button button button--action button--gradient-violet")
choose_game_btn = driver.find_element(By.CSS_SELECTOR, ".header__menu-button")
choose_game_btn.click()

import time
time.sleep(2)  # Adjust if needed

menu_content = driver.find_element(By.CLASS_NAME, "menu__wrapper").get_attribute('innerHTML')
print(menu_content)

driver.quit()


<div data-v-013755fa="" class="menu"><div data-v-013755fa="" class="menu__container container"><div data-v-013755fa="" class="menu__header"><!----> <div data-v-013755fa="" class="menu__logo-container"><img data-v-013755fa="" src="/images/logos/rebranded/logo.svg" alt="Skycoach" class="menu__logo"></div> <button data-v-f2586174="" data-v-013755fa="" class="menu__header-close-button button button--action"><svg data-v-039e196d="" data-v-013755fa="" aria-hidden="true" class="ui-icon" data-v-f2586174=""><use data-v-039e196d="" href="/images/icons/sprite.svg?v=14#x"></use></svg></button> <div data-v-04c8ddfb="" data-v-013755fa="" class="menu__search-mode-container menu__search-mode"><div data-v-281ef3c4="" data-v-04c8ddfb="" class="menu__search-mode-input input-container"><!----> <div data-v-281ef3c4="" class="input-background input-background--with-icon"><svg data-v-039e196d="" data-v-281ef3c4="" aria-hidden="true" class="ui-icon input-icon"><use data-v-039e196d="" href="/images/icons/sprit

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

soup = BeautifulSoup(menu_content, "html.parser")
games = []

for li in soup.find_all("li", class_="menu__game-section-item"):
    a_tag = li.find("a", class_="menu__game-section-item-link")
    if a_tag:
        name = a_tag.find("span").get_text(strip=True)
        href = a_tag.get("href")
        img_tag = a_tag.find("img", class_="menu__game-section-item-image")
        img_src = img_tag.get("src") if img_tag else None
        games.append({"Game Name": name, "Link": href, "Image": img_src})

df_games = pd.DataFrame(games)
print(df_games)


In [ ]:
df_games.to_csv("games.csv", index=False)


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import time
import os
import requests
from bs4 import BeautifulSoup
options = Options()
options.add_argument('--headless')
driver = webdriver.Edge(options=options)

# Load the CSV file and display its contents
df = pd.read_csv("games.csv")

all_services = []

for index, row in df.iterrows():
    game_name = row['Game Name']
    game_link = row['Link']
    print(f"Fetching data for {game_name} from {game_link}")
    if ":" in game_name:
        game_name = game_name.replace(":", "-")
    complete_game_link = "https://skycoach.gg" + game_link
    print(f"Complete game link: {complete_game_link}")
    # Fetch the HTML content of the game page
    page_source = requests.get(complete_game_link).text
    time.sleep(2)  # Wait for the page to load

    game_page_content = page_source
    soup = BeautifulSoup(game_page_content, "html.parser")
    sidebar = soup.find("div", class_="game-tags-sidebar__categories")
    if sidebar:
        # Find all main categories
        for cat in sidebar.find_all("div", class_="game-tags-sidebar__category", recursive=True):
            head = cat.find("div", class_="game-tags-sidebar__category-head")
            print(head)
            if head:
                a_tag = head.find("a", class_="game-tags-sidebar__category-title")
                if a_tag:
                    print(a_tag)
                    service_name = a_tag.get_text(strip=True)
                    service_link = a_tag.get("href")
                    all_services.append({
                        "Game Name": game_name,
                        "Category": service_name,
                        "Link": "https://skycoach.gg" + service_link if service_link else ""
                    })
            # Check for subcategories
            subcats = cat.find("div", class_="game-tags-sidebar__subcategories")
            if subcats:
                print(subcats)
                for sub_a in subcats.find_all("a", class_="game-tags-sidebar__subcategory"):
                    sub_name = sub_a.get_text(strip=True)
                    sub_link = sub_a.get("href")
                    all_services.append({
                        "Game Name": game_name,
                        "Category": service_name,
                        "Subcategory": sub_name,
                        "Link": "https://skycoach.gg" + sub_link if sub_link else ""
                    })
    print(f"Processed sidebar for {game_name}")
    df_services = pd.DataFrame(all_services)
    # check if folder exists
    if not os.path.exists("gamesTwo"):
        os.makedirs("gamesTwo")
    df_services.to_csv(f"gamesTwo/{game_name.replace(' ', '_')}_services.csv", index=False)
    all_services = []  # Reset for the next game
    time.sleep(2)  # Adjust sleep time as needed
    
    # Remove break if you want to process all games


# Convert to DataFrame and save to CSV
    

In [ ]:
import json

api_url = "https://skycoach.gg/api/shop/product/list/badge-products?badge_id=3&fields_data=compact"
response = requests.get(api_url)
data = response.json()
data_keys = data.keys()
print(data)
# Create a sample payload structure with "" values for each field
# sample_payload = {}
# for key in data_keys:
#     if isinstance(data[key], list) and data[key]:
#         sample_payload[key] = {field: "" for field in data[key][0].keys()}
#     else:
#         sample_payload[key] = None
# print(data)

# Save the sample payload to a JSON file
# with open("sample_payload.json", "w") as f:
#     json.dump(sample_payload, f, indent=4)

# save data in text file.txt
with open("data.txt", "w") as f:
    json.dump(data, f, indent=4)

# print(sample_payload)

for key in data_keys:
    print(f"Key: {key}")
    for item in data[key]:
        title = item.get("title", "No title")
        price = item.get("price", "No price")
        print(f"Title: {title}, Price: {price}")
# for item in data["2"]:
#     title = item.get("title", "No title")
#     price = item.get("price", "No price")
#     print(f"Title: {title}, Price: {price}")

In [ ]:
import os

games_files = set(os.listdir("games"))
gamesTwo_files = set(os.listdir("gamesTwo"))

# Find files present in gamesTwo but not in games
extra_in_gamesTwo = gamesTwo_files - games_files
# Find files present in games but not in gamesTwo
extra_in_games = games_files - gamesTwo_files

print("Extra files in gamesTwo:")
for fname in extra_in_gamesTwo:
    print(fname)

print("\nExtra files in games:")
for fname in extra_in_games:
    print(fname)

In [ ]:
# https://www.skycoach.gg/storage/uploads/game-assets/icons/8a9fa179ac.svg

# save this url svg in local directory /uploads
def save_svg_image(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
# Example usage
save_svg_image("https://www.skycoach.gg/storage/uploads/game-assets/icons/254932daa2.svg", "game_icon.svg")

In [ ]:
import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.edge.options import Options
# import time
import os
# import requests
# from bs4 import BeautifulSoup
# options = Options()
# options.add_argument('--headless')
# driver = webdriver.Edge(options=options)
# /get all file names from gamesTwo directory
gamesTwo_files = os.listdir("gamesTwo")


# Loop through each file in the directory
for file_name in gamesTwo_files:
    file_name = "RBLX_services.csv"
    if file_name.endswith(".csv"):
        # Read the CSV file
        df = pd.read_csv(f"./gamesTwo/{file_name}")
        
        # Check if the 'Link' column exists
        if 'Link' in df.columns:
            # Loop through each row in the DataFrame
            for index, row in df.iterrows():
                link = row['Link']
                if link and link.startswith("https://skycoach.gg/"):
                    # extract data from link
                    print(f"Processing link: {link}")
                    # Here you can add your logic to process the link
                    page_source = requests.get(link).text
                    print(page_source)
                    """<div class="game-header" data-v-265d981e data-v-059b5164><div class="game-header__top-container" data-v-265d981e><h1 class="h1 sm:h1-mobile" data-v-265d981e>
      8 Ball Pool Coins
    </h1> <!----></div> <!----></div>
    
    <picture class="responsive-image offer-card__image" data-v-26f85f9a data-v-0d6f92b4><source media="(-webkit-min-device-pixel-ratio: 2)" type="" srcset="/storage/uploads/products/8-ball-pool-coins1717452870_picture_item.png" data-v-26f85f9a> <img src="/storage/uploads/products/8-ball-pool-coins1717452871_picture_item_small.png" data-fallback-img="/images/games/picture_item.png" class="responsive-image__img" data-v-26f85f9a></picture> 
 
       <div class="product-info-sections product-offer__description" data-v-3d474cdb data-v-059b5164><div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      Description
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><p><span>Buy cheap 8 Ball Pool Coins&nbsp;on our platform and be ahead of everyone else.</span></p>
<p><span>In 8 Ball Pool, players can use special props like more accurate aiming lines and power-enhancing items. By acquiring enough 8 Ball Pool coins, players can purchase these props to gain additional advantages and improve their gameplay. So, there is never enough of this valuable currency, and getting it is a very tiring and time-consuming process. So, why waste your precious time when you can quickly get our 8 Ball Pool Coins for sale?</span></p>
<p><span>Contact us now and get as many 8 Ball Pool Coins as you need with maximum safety and quick delivery!</span></p>
<ol>
<li><span>Selected amount of 8 Ball Pool Coins;</span></li>
<li><span>100% safety;</span></li>
<li><span>Complete safety and 24/7 support.</span></li>
</ol></div></div></div></div> <div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      Requirements
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><ul>
<li>Active account;</li>
<li>20+ level (Semi-pro Rank).</li>
</ul></div></div></div></div> <div class="product-info-section" data-v-3d474cdb><h2 class="product-info-section__title h2" data-v-3d474cdb>
      More info
    </h2> <div class="product-info-section__content" data-v-3d474cdb><div class="product-info-section__wrapper" data-v-3d474cdb><div class="product-info-section__html text-medium wysiwyg-content" data-v-3d474cdb><p><span>HOW DO WE DELIVER 8 BALL POOL COINS?</span></p>
<p>You can get cheap 8 Ball Pool Coins via the following completion method:</p>
<p><span>Comfort Trade:</span></p>
<ul>
<li><span>This is the quickest and the most reliable 8 Ball Pool Coins delivery method. Our pro-player will safely access your account and carry out your order. All security measures will be in constant use so that you can rest assured - your account is in good hands.&nbsp;</span></li>
</ul></div></div></div></div></div>



<a href="/8-ball-pool-boost" class="breadcrumbs__back-button router-link-active button button--secondary button--white-transparent-10" data-v-f2586174 data-v-a5397112><svg aria-hidden="true" class="ui-icon breadcrumbs__back-button-icon" data-v-039e196d data-v-a5397112><use href="/images/icons/sprite.svg?v=14#chevron-left" data-v-039e196d></use></svg> <span class="breadcrumbs__back-button-text" data-v-a5397112>Coins</span></a>
 
<div class="payment-summary__price" data-v-0ce13794><!----> <div class="payment-summary__price-row payment-summary__price-row-total" data-v-0ce13794><div class="payment-summary__label-column" data-v-0ce13794><span class="payment-summary__label-total text-medium" data-v-0ce13794>
          Total
        </span> <!----></div> <div class="payment-summary__price-column" data-v-0ce13794><!----> <span class="payment-summary__price-column-total h2" data-v-0ce13794>
          0,99 €
        </span></div></div></div>     
       
             """

    break

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
results = []
gamesTwo_files = list(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: {link}")
          page_source = requests.get(link).text
          soup = BeautifulSoup(page_source, "html.parser")
          
          # Name
          name_tag = soup.find("div", class_="game-header")
          name = None
          if name_tag:
            h1 = name_tag.find("h1")
            if h1:
              name = h1.get_text(strip=True)
          
          # Description
          desc = None
          desc_section = soup.find("div", class_="product-info-section__html")
          if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
          
          # Icon
          icon = None
          picture = soup.find("picture", class_="responsive-image")
          if picture:
            img = picture.find("img")
            if img and img.get("src"):
              icon = img["src"]
              if icon.startswith("/"):
                icon = "https://skycoach.gg" + icon
          
          # Price
          price = None
          price_span = soup.find("span", class_="payment-summary__price-column-total")
          if price_span:
            price = price_span.get_text(strip=True)
          
          # Category
          cat = row.get("Category", category)
          
          results.append({
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": link
          })
    break

df_result = pd.DataFrame(results)
print(df_result)


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: https://skycoach.gg/roblox-boost{link}")
          page_source = requests.get("https://skycoach.gg/roblox-boost").text
          soup = BeautifulSoup(page_source, "html.parser")
          
        #   # Name
        #   name_tag = soup.find("div", class_="game-header")
        #   name = None
        #   if name_tag:
        #     h1 = name_tag.find("h1")
        #     if h1:
        #       name = h1.get_text(strip=True)
          
        #   # Description
        #   desc = None
        #   desc_section = soup.find("div", class_="product-info-section__html")
        #   if desc_section:
        #     desc = desc_section.get_text(separator="\n", strip=True)
          
        #   # Icon
        #   icon = None
        #   picture = soup.find("picture", class_="responsive-image")
        #   if picture:
        #     img = picture.find("img")
        #     if img and img.get("src"):
        #       icon = img["src"]
        #       if icon.startswith("/"):
        #         icon = "https://skycoach.gg" + icon
          
        #   # Price
        #   price = None
        #   price_span = soup.find("span", class_="payment-summary__price-column-total")
        #   if price_span:
        #     price = price_span.get_text(strip=True)
          
        #   # Category
        #   cat = row.get("Category", category)
          
        #   results.append({
        #     "Name": name,
        #     "Description": desc,
        #     "Icon": icon,
        #     "Price": price,
        #     "Category": cat,
        #     "Link": link
        #   })
    break

df_result = pd.DataFrame(results)
print(df_result)


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
  if file_name.endswith(".csv"):
    print(file_name)
    df = pd.read_csv(f"gamesTwo/{file_name}")
    category = file_name.replace("_services.csv", "").replace("_", " ")
    if 'Link' in df.columns:
      for index, row in df.iterrows():
        print(row)
        subcat = row.get("Subcategory", "")
        if subcat:
          print(f"Processing subcategory: {subcat}")
        else:
          print(f"Processing category: {category}")
        link = row['Link']
        if link and link.startswith("https://skycoach.gg/"):
          print(f"Processing link: {link}")
          page_source = requests.get("https://skycoach.gg/path-of-exile-2-boost/products/exalted-orbs-6477").text
          soup = BeautifulSoup(page_source, "html.parser")
          # asides = soup.find_all("aside", class_="game-tags-sidebar game-main-page-sidebar")
          # soup2 = BeautifulSoup(str(asides), "html.parser")
          # hrefs = [a.get("href") for a in soup2.find_all("a") if a.get("href")]

          # Name
          name_tag = soup.find("div", class_="game-header")
          name = None
          if name_tag:
            h1 = name_tag.find("h1")
            if h1:
              name = h1.get_text(strip=True)
          
          # Description
          desc = None
          desc_section = soup.find("div", class_="product-info-section__html")
          if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
          
          # Icon
          icon = None
          picture = soup.find("picture", class_="responsive-image")
          if picture:
            img = picture.find("img")
            if img and img.get("src"):
              icon = img["src"]
              if icon.startswith("/"):
                icon = "https://skycoach.gg" + icon
          
          # Price
          price = None
          price_span = soup.find("span", class_="payment-summary__price-column-total")
          if price_span:
            price = price_span.get_text(strip=True)
          
          # Category
          cat = row.get("Category", category)
          
          results.append({
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": link
          })
    break


print(hrefs)


Monster_Hunter_Wilds_services.csv
Game Name                                 Monster Hunter Wilds
Category                                         Powerleveling
Link         https://skycoach.gg/monster-hunter-wilds-boost...
Name: 0, dtype: object
Processing category: Monster Hunter Wilds
Processing link: https://skycoach.gg/monster-hunter-wilds-boost/powerleveling
Game Name                                 Monster Hunter Wilds
Category                                               Gearing
Link         https://skycoach.gg/monster-hunter-wilds-boost...
Name: 1, dtype: object
Processing category: Monster Hunter Wilds
Processing link: https://skycoach.gg/monster-hunter-wilds-boost/gearing
Game Name                                 Monster Hunter Wilds
Category                                       Account Driving
Link         https://skycoach.gg/monster-hunter-wilds-boost...
Name: 2, dtype: object
Processing category: Monster Hunter Wilds
Processing link: https://skycoach.gg/monster-hunter-wi

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
options = Options()
# options.add_argument('--headless')  # Uncomment if you want to run in headless
count = 0

driver = webdriver.Edge(options=options)
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
    if file_name.endswith(".csv"):
        existing_file = f"skycoach_results_{file_name}.csv"
        existing_file_folder = os.listdir("game_services")
        if existing_file in existing_file_folder:
            print(f"Skipping {file_name} as results already exist.")
            continue
        df = pd.read_csv(f"gamesTwo/{file_name}")
        category = file_name.replace("_services.csv", "").replace("_", " ")
        if 'Link' in df.columns:
            for index, row in df.iterrows():
                subcat = row.get("Subcategory", "")
                if subcat:
                    print(f"Processing subcategory: {subcat}")
                else:
                    print(f"Processing category: {category}")
                link = row['Link']
                if link and link.startswith("https://skycoach.gg/"):
                    print(f"Processing link: {link}")
                    # Use selenium driver to get page source
                    driver.get(link)
                    page_source = driver.page_source
                    soup = BeautifulSoup(page_source, "html.parser")
                    # Find all li elements inside the specified class and print their hrefs
                    container = soup.find("div", class_="card-list game-tag-page__container game-tag-page__products-list")
                    nestedlinks = []
                    if container:
                        li_tags = container.find_all("li")
                        for li in li_tags:
                            a_tag = li.find("a", href=True)
                            if a_tag:
                                href = a_tag["href"]
                                nestedlinks.append(href)
                    print(nestedlinks)
                    # Name
                    if nestedlinks:
                        for  nestedlink in nestedlinks:
                            if nestedlink.startswith("/"):
                                nestedlink = "https://skycoach.gg" + nestedlink
                        # Fetch the first link from nested links
                                print(f"Fetching details from: {nestedlink}")
                                driver.get(f'{nestedlink}')
                                time.sleep(2)
                                page_source = driver.page_source
                                soup = BeautifulSoup(page_source, "html.parser")
                                name_tag = soup.find("div", class_="game-header")
                                name = None
                                if name_tag:
                                    h1 = name_tag.find("h1")
                                    if h1:
                                        name = h1.get_text(strip=True)

                                # Description
                                desc = None
                                desc_section = soup.find("div", class_="product-info-section__html")
                                if desc_section:
                                    desc = desc_section.get_text(separator="\n", strip=True)

                                # Icon
                                icon = None
                                picture = soup.find("picture", class_="responsive-image")
                                if picture:
                                    img = picture.find("img")
                                    if img and img.get("src"):
                                        icon = img["src"]
                                        if icon.startswith("/"):
                                            icon = "https://skycoach.gg" + icon

                                # Price
                                price = None
                                price_span = soup.find("span", class_="payment-summary__price-column-total")
                                if price_span:
                                    price = price_span.get_text(strip=True)

                                # Category
                                cat = row.get("Category", category)

                                results.append({
                                    "Name": name,
                                    "Description": desc,
                                    "Icon": icon,
                                    "Price": price,
                                    "Category": cat,
                                    "Link": link
                                }) 
                                count += 1
                                print(count)
                
        
    df_result = pd.DataFrame(results)
    df_result.to_csv(f"skycoach_results_{file_name}.csv", index=False)
    df_result = []
print(df_result)


KeyboardInterrupt: 

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

options = Options()
options.page_load_strategy = 'eager'
# options.add_argument('--headless')  # Uncomment if you want to run in headless

def process_nestedlink(nestedlink, row, category):
    driver = webdriver.Edge(options=options)
    result = {}
    try:
        if nestedlink.startswith("/"):
            nestedlink = "https://skycoach.gg" + nestedlink
        driver.get(nestedlink)
        time.sleep(2)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        name_tag = soup.find("div", class_="game-header")
        name = None
        if name_tag:
            h1 = name_tag.find("h1")
            if h1:
                name = h1.get_text(strip=True)
        desc = None
        desc_section = soup.find("div", class_="product-info-section__html")
        if desc_section:
            desc = desc_section.get_text(separator="\n", strip=True)
        icon = None
        picture = soup.find("picture", class_="responsive-image")
        if picture:
            img = picture.findAll("img")
            # (old code, not needed for icon extraction)
        price = None
        price_span = soup.find("span", class_="payment-summary__price-column-total")
        if price_span:
            price = price_span.get_text(strip=True)
        cat = row.get("Category", category)

        # Extract game character image
        game_char_image = None
        image_container = soup.find("div", class_="offer-card__image-container")
        if image_container:
            picture_tag = image_container.find("picture", class_="responsive-image offer-card__image")
            if picture_tag:
                # Try to get src from <source> first, then <img>
                src = None
                source_tag = picture_tag.find("source")
                img_tag = picture_tag.find("img")
                if source_tag and source_tag.has_attr("srcset"):
                    src = source_tag["srcset"]
                elif img_tag and img_tag.has_attr("src"):
                    src = img_tag["src"]
                if src and src.startswith("/"):
                    src = "https://skycoach.gg" + src
                game_char_image = src

        result = {
            "Name": name,
            "Description": desc,
            "Icon": icon,
            "Price": price,
            "Category": cat,
            "Link": row['Link'],
            "GameCharImage": game_char_image
        }
    finally:
        driver.quit()
    return result


count = 0
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
    if file_name.endswith(".csv"):
        existing_file = f"skycoach_results_{file_name}.csv"
        existing_file_folder = os.listdir("game_services")
        if existing_file in existing_file_folder:
            print(f"Skipping {file_name} as results already exist.")
            continue
        df = pd.read_csv(f"gamesTwo/{file_name}")
        category = file_name.replace("_services.csv", "").replace("_", " ")
        if 'Link' in df.columns:
            with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers as needed
                futures = []
                for index, row in df.iterrows():
                    subcat = row.get("Subcategory", "")
                    if subcat:
                        print(f"Processing subcategory: {subcat}")
                    else:
                        print(f"Processing category: {category}")
                    link = row['Link']
                    if link and link.startswith("https://skycoach.gg/"):
                        print(f"Processing link: {link}")
                        # Use a temporary driver to get nested links
                        driver = webdriver.Edge(options=options)
                        driver.get(link)
                        page_source = driver.page_source
                        soup = BeautifulSoup(page_source, "html.parser")
                        container = soup.find("div", class_="card-list game-tag-page__container game-tag-page__products-list")
                        nestedlinks = []
                        if container:
                            li_tags = container.find_all("li")
                            for li in li_tags:
                                a_tag = li.find("a", href=True)
                                if a_tag:
                                    href = a_tag["href"]
                                    nestedlinks.append(href)
                        driver.quit()
                        print(nestedlinks)
                        for nestedlink in nestedlinks:
                            futures.append(executor.submit(process_nestedlink, nestedlink, row, category))
                for future in as_completed(futures):
                    result = future.result()
                    if result:
                        results.append(result)
                        count += 1
                        print(count)
        df_result = pd.DataFrame(results)
        df_result.to_csv(f"skycoach_results_{file_name}.csv", index=False)
        results = []
print("Done.")


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


Processing subcategory: nan
Processing link: https://skycoach.gg/diablo-4-boost/season
['/diablo-4-boost/gold-coins', '/diablo-4-boost/products/powerleveling-2099', '/diablo-4-boost/products/paragon-leveling-2204', '/diablo-4-boost/products/diablo-4-belial-kill-7956', '/diablo-4-boost/products/runewords-5937', '/diablo-4-boost/products/lair-boss-kill-7961', '/diablo-4-boost/products/lair-boss-key-packs-7963', '/diablo-4-boost/products/leveling-bundles-8012', '/diablo-4-boost/products/horadric-jewels-8253', '/diablo-4-boost/products/dante-pet-8254', '/diablo-4-boost/products/horadric-phials-8256', '/diablo-4-boost/products/horadric-spell-8258', '/diablo-4-boost/products/horadric-knowledge-rank-8264', '/diablo-4-boost/products/diablo-4-unique-gear-boost-2400', '/diablo-4-boost/products/lair-boss-keys-7962', '/diablo-4-boost/products/mythic-runewords-bundle-7994', '/diablo-4-boost/products/the-pit-of-artificers-4592', '/diablo-4-boost/products/torment-difficulties-5948', '/diablo-4-boost/

There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


[]
Processing subcategory: nan
Processing link: https://skycoach.gg/diablo-4-boost/bosses


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
/tmp/ipykernel_373569/3319955046.py:40: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  img = picture.findAll("img")
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using dr

...

There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


['/diablo-4-boost/products/powerleveling-2099', '/diablo-4-boost/products/paragon-leveling-2204', '/diablo-4-boost/products/leveling-bundles-8012', '/diablo-4-boost/products/torment-difficulties-5948', '/diablo-4-boost/products/codex-of-power-boost-4817', '/diablo-4-boost/products/diablo-4-season-journey-2456', '/diablo-4-boost/products/seasonal-campaign-boost-3965', '/diablo-4-boost/products/diablo-4-season-starter-pack-2457', '/diablo-4-boost/products/hardcore-powerleveling-2347', '/diablo-4-boost/products/diablo-4-story-mode-2206', '/diablo-4-boost/products/glyphs-leveling-2415', '/diablo-4-boost/products/diablo-4-the-heroic-roster-achievement-2739']
Processing subcategory: nan
Processing link: https://skycoach.gg/diablo-4-boost/gearing


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


['/diablo-4-boost/products/runewords-5937', '/diablo-4-boost/products/horadric-jewels-8253', '/diablo-4-boost/products/horadric-phials-8256', '/diablo-4-boost/products/horadric-spell-8258', '/diablo-4-boost/products/diablo-4-unique-gear-boost-2400', '/diablo-4-boost/products/mythic-runewords-bundle-7994', '/diablo-4-boost/products/mythic-unique-gear-5833', '/diablo-4-boost/products/codex-of-power-boost-4817', '/diablo-4-boost/products/infernal-compass-5490', '/diablo-4-boost/products/diablo-4-legendary-items-2373', '/diablo-4-boost/products/helltide-farming-4805', '/diablo-4-boost/products/build-check-7245', '/diablo-4-boost/products/diablo-4-gems-farm-2422', '/diablo-4-boost/products/heir-of-perdition-5787', '/diablo-4-boost/products/shroud-of-false-death-5790', '/diablo-4-boost/products/nesekem-the-herald-5791', '/diablo-4-boost/products/bis-ancestral-gear-2414', '/diablo-4-boost/products/resplendent-sparks-farm-4139', '/diablo-4-boost/products/ugly-bastard-helm-5825', '/diablo-4-boo

There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache
There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in th

In [2]:
import csv
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import time
from datetime import datetime

def parse_price_modifier(price_text):
    """Extract numeric price value from text like '+6,43 €' or 'Free'"""
    if not price_text or price_text.lower() in ['free', 'basic']:
        return 0.00
    
    # Remove currency symbols and extract number
    price_clean = re.sub(r'[^\d,.-]', '', price_text)
    price_clean = price_clean.replace(',', '.')
    
    try:
        return float(price_clean)
    except ValueError:
        return 0.00

def clear_csv_file(filename='service_options.csv'):
    """Clear the CSV file and create a new one with just headers"""
    fieldnames = [
        'option_id', 'service_id', 'parent_option_id', 'option_type', 'option_name',
        'option_label', 'option_value', 'price_modifier', 'min_value', 'max_value',
        'default_value', 'is_required', 'display_order', 'is_active', 'created_at', 'updated_at'
    ]
    
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
    print(f"Cleared CSV file: {filename}")

def get_next_option_id(filename='service_options.csv'):
    """Get the next available option_id from existing CSV file"""
    import os
    if not os.path.exists(filename):
        return 1
    
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            max_id = 0
            for row in reader:
                try:
                    option_id = int(row['option_id'])
                    max_id = max(max_id, option_id)
                except (ValueError, KeyError):
                    continue
            return max_id + 1
    except Exception:
        return 1

def get_next_service_id(filename='service_options.csv'):
    """Get the next available service_id from existing CSV file"""
    import os
    if not os.path.exists(filename):
        return 1
    
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            max_id = 0
            for row in reader:
                try:
                    service_id = int(row['service_id'])
                    max_id = max(max_id, service_id)
                except (ValueError, KeyError):
                    continue
            return max_id + 1
    except Exception:
        return 1

def extract_service_options(html_content, service_id=None, start_option_id=None):
    """Extract service options from HTML content"""
    soup = BeautifulSoup(html_content, 'html.parser')
    options = []
    
    # Get service ID - auto-increment if not provided
    if service_id is None:
        service_id = get_next_service_id()
    
    # Get starting option ID
    if start_option_id is None:
        option_id_counter = get_next_option_id()
    else:
        option_id_counter = start_option_id
        
    display_order = 1
    
    # Find the main options container
    options_container = soup.find('div', class_='product-detail-calculator__options')
    if not options_container:
        return options
    
    option_groups = options_container.find_all('div', class_='option-group')
    
    for group in option_groups:
        product_option = group.find('div', class_='product-option')
        if not product_option:
            continue
            
        # Get option label from header
        option_head = product_option.find('div', class_='product-option__head')
        option_label = ""
        if option_head:
            label_div = option_head.find('div', class_='product-option__label')
            if label_div:
                option_label = label_div.get_text(strip=True).replace(':', '')
        
        # Check for range/slider input
        range_cluster = product_option.find('div', class_='product-option-cluster-range')
        if range_cluster:
            # Handle range/slider options
            input_containers = range_cluster.find_all('div', class_='input-container')
            
            for idx, container in enumerate(input_containers):
                label_div = container.find('div', class_='label')
                input_tag = container.find('input')
                
                if label_div and input_tag:
                    input_label = label_div.get_text(strip=True)
                    default_val = input_tag.get('value', '')
                    
                    # Extract min/max from range scale if available
                    min_val, max_val = None, None
                    range_container = range_cluster.find('div', class_='range-container')
                    if range_container:
                        scale_items = range_container.find_all('div', class_='range__scale-item')
                        if scale_items:
                            try:
                                min_val = int(scale_items[0].get_text(strip=True))
                                max_val = int(scale_items[-1].get_text(strip=True))
                            except ValueError:
                                pass
                    
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': None,
                        'option_type': 'slider',
                        'option_name': f"{option_label.lower().replace(' ', '_')}_{input_label.lower().replace(' ', '_')}",
                        'option_label': input_label,
                        'option_value': default_val,
                        'price_modifier': 0.00,
                        'min_value': min_val,
                        'max_value': max_val,
                        'default_value': default_val,
                        'is_required': 1,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1
        
        # Check for dropdown/select
        select_cluster = product_option.find('div', class_='product-option-cluster-select')
        if select_cluster:
            select_tag = select_cluster.find('select')
            if select_tag:
                parent_id = option_id_counter
                
                # Create parent option for dropdown
                options.append({
                    'option_id': parent_id,
                    'service_id': service_id,
                    'parent_option_id': None,
                    'option_type': 'dropdown',
                    'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                    'option_label': option_label,
                    'option_value': None,
                    'price_modifier': 0.00,
                    'min_value': None,
                    'max_value': None,
                    'default_value': None,
                    'is_required': 1,
                    'display_order': display_order,
                    'is_active': 1,
                    'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
                option_id_counter += 1
                display_order += 1
                
                # Add dropdown options
                dropdown_options = select_tag.find_all('option')
                for opt in dropdown_options:
                    option_text = opt.get_text(strip=True)
                    option_val = opt.get('value', '')
                    
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'dropdown',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': 0.00,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if opt.get('selected') else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1
        
        # Check for radio buttons
        radio_cluster = product_option.find('div', class_='product-option-cluster-radios')
        if radio_cluster:
            parent_id = option_id_counter
            
            # Create parent option for radio group
            options.append({
                'option_id': parent_id,
                'service_id': service_id,
                'parent_option_id': None,
                'option_type': 'radio',
                'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                'option_label': option_label,
                'option_value': None,
                'price_modifier': 0.00,
                'min_value': None,
                'max_value': None,
                'default_value': None,
                'is_required': 1,
                'display_order': display_order,
                'is_active': 1,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            option_id_counter += 1
            display_order += 1
            
            # Add radio options
            radio_options = radio_cluster.find_all('div', class_='radio-option')
            for radio_opt in radio_options:
                input_tag = radio_opt.find('input', {'type': 'radio'})
                label_span = radio_opt.find('span', class_='radio-check__label')
                price_div = radio_opt.find('div', class_='radio-option__price')
                
                if input_tag and label_span:
                    # Get the label text more carefully - it might have nested elements
                    label_text_parts = []
                    for text_node in label_span.stripped_strings:
                        if text_node.strip() and not text_node.strip().startswith('+') and not text_node.strip().lower() in ['free']:
                            label_text_parts.append(text_node.strip())
                            break  # Take only the first meaningful text
                    
                    option_text = label_text_parts[0] if label_text_parts else label_span.get_text(strip=True).split('\n')[0].strip()
                    option_val = input_tag.get('value', '')
                    is_checked = input_tag.get('checked') is not None
                    
                    price_modifier = 0.00
                    if price_div:
                        price_text = price_div.get_text(strip=True)
                        price_modifier = parse_price_modifier(price_text)
                    
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'radio',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': price_modifier,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if is_checked else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1
        
        # Check for checkboxes
        checkbox_cluster = product_option.find('div', class_='product-option-cluster-checkboxes')
        if checkbox_cluster:
            parent_id = option_id_counter
            
            # Create parent option for checkbox group
            options.append({
                'option_id': parent_id,
                'service_id': service_id,
                'parent_option_id': None,
                'option_type': 'checkbox',
                'option_name': option_label.lower().replace(' ', '_').replace(':', ''),
                'option_label': option_label,
                'option_value': None,
                'price_modifier': 0.00,
                'min_value': None,
                'max_value': None,
                'default_value': None,
                'is_required': 0,
                'display_order': display_order,
                'is_active': 1,
                'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
            option_id_counter += 1
            display_order += 1
            
            # Add checkbox options
            checkbox_options = checkbox_cluster.find_all('div', class_='checkbox-option')
            for checkbox_opt in checkbox_options:
                input_tag = checkbox_opt.find('input', {'type': 'checkbox'})
                label_span = checkbox_opt.find('span', class_='radio-check__label')
                price_div = checkbox_opt.find('div', class_='checkbox-option__price')
                
                if input_tag and label_span:
                    option_text = label_span.get_text(strip=True)
                    option_val = input_tag.get('value', '')
                    is_checked = input_tag.get('checked') is not None
                    
                    price_modifier = 0.00
                    if price_div:
                        price_text = price_div.get_text(strip=True)
                        price_modifier = parse_price_modifier(price_text)
                    
                    options.append({
                        'option_id': option_id_counter,
                        'service_id': service_id,
                        'parent_option_id': parent_id,
                        'option_type': 'checkbox',
                        'option_name': f"{option_label.lower().replace(' ', '_').replace(':', '')}_{option_text.lower().replace(' ', '_').replace('%', 'percent')}",
                        'option_label': option_text,
                        'option_value': option_val,
                        'price_modifier': price_modifier,
                        'min_value': None,
                        'max_value': None,
                        'default_value': option_val if is_checked else None,
                        'is_required': 0,
                        'display_order': display_order,
                        'is_active': 1,
                        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
                    option_id_counter += 1
                    display_order += 1
    
    return options

def save_options_to_csv(options, filename='service_options.csv', append_mode=True):
    """Save options to CSV file"""
    if not options:
        print("No options to save")
        return
    
    fieldnames = [
        'option_id', 'service_id', 'parent_option_id', 'option_type', 'option_name',
        'option_label', 'option_value', 'price_modifier', 'min_value', 'max_value',
        'default_value', 'is_required', 'display_order', 'is_active', 'created_at', 'updated_at'
    ]
    
    import os
    file_exists = os.path.exists(filename)
    
    if append_mode and file_exists:
        # Append mode: add to existing file without header
        with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerows(options)
        print(f"Appended {len(options)} options to {filename}")
    else:
        # Write mode: create new file or overwrite with header
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(options)
        print(f"Saved {len(options)} options to {filename}")

def scrape_multiple_services(urls, clear_csv_first=False):
    """
    Scrape multiple service URLs with auto-incrementing service IDs
    
    Args:
        urls: List of URLs to scrape
        clear_csv_first: If True, clears the CSV file before starting
    
    Returns:
        List of tuples: [(service_id, options_count, url), ...]
    """
    if clear_csv_first:
        clear_csv_file()
    
    results = []
    
    for i, url in enumerate(urls):
        print(f"\\n=== Processing URL {i+1}/{len(urls)} ===")
        options = scrape_service_options(url, append_to_csv=True)
        
        if options:
            service_id = options[0]['service_id']
            results.append((service_id, len(options), url))
        else:
            results.append((None, 0, url))
        
        # Small delay between requests
        time.sleep(1)
    
    print(f"\\n=== Summary ===")
    for service_id, count, url in results:
        if service_id:
            print(f"Service {service_id}: {count} options from {url}")
        else:
            print(f"Failed to extract options from {url}")
    
    return results

def scrape_service_options(url, service_id=None, append_to_csv=True):
    """Main function to scrape service options from a URL"""
    options = webdriver.EdgeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    driver = webdriver.Edge(options=options)
    
    # Get service ID - auto-increment if not provided
    if service_id is None:
        service_id = get_next_service_id()
    
    try:
        if url.startswith("/"):
            url = "https://skycoach.gg" + url
        
        print(f"Scraping options from: {url} (Service ID: {service_id})")
        driver.get(url)
        time.sleep(3)  # Wait for page to load
        
        page_source = driver.page_source
        options_data = extract_service_options(page_source, service_id)
        
        if options_data:
            save_options_to_csv(options_data, append_mode=append_to_csv)
            print(f"Successfully extracted {len(options_data)} options for service {service_id}")
            return options_data
        else:
            print("No options found on the page")
            return []
            
    except Exception as e:
        print(f"Error scraping options: {str(e)}")
        return []
    finally:
        driver.quit()


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

options = Options()
options.page_load_strategy = 'eager'
# options.add_argument('--headless')  # Uncomment if you want to run in headless

def process_nestedlink(nestedlink, row, category):
    driver = webdriver.Edge(options=options)
    result = {}
    try:
        if nestedlink.startswith("/"):
            nestedlink = "https://skycoach.gg" + nestedlink
        print(nestedlink)
        driver.get(nestedlink)
        time.sleep(2)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        return
        # name_tag = soup.find("div", class_="game-header")
        # name = None
        # if name_tag:
        #     h1 = name_tag.find("h1")
        #     if h1:
        #         name = h1.get_text(strip=True)
        # desc = None
        # desc_section = soup.find("div", class_="product-info-section__html")
        # if desc_section:
        #     desc = desc_section.get_text(separator="\n", strip=True)
        # icon = None
        # picture = soup.find("picture", class_="responsive-image")
        # if picture:
        #     img = picture.findAll("img")
        #     # (old code, not needed for icon extraction)
        # price = None
        # price_span = soup.find("span", class_="payment-summary__price-column-total")
        # if price_span:
        #     price = price_span.get_text(strip=True)
        # cat = row.get("Category", category)

        # # Extract game character image
        # game_char_image = None
        # image_container = soup.find("div", class_="offer-card__image-container")
        # if image_container:
        #     picture_tag = image_container.find("picture", class_="responsive-image offer-card__image")
        #     if picture_tag:
        #         # Try to get src from <source> first, then <img>
        #         src = None
        #         source_tag = picture_tag.find("source")
        #         img_tag = picture_tag.find("img")
        #         if source_tag and source_tag.has_attr("srcset"):
        #             src = source_tag["srcset"]
        #         elif img_tag and img_tag.has_attr("src"):
        #             src = img_tag["src"]
        #         if src and src.startswith("/"):
        #             src = "https://skycoach.gg" + src
        #         game_char_image = src

        # result = {
        #     "Name": name,
        #     "Description": desc,
        #     "Icon": icon,
        #     "Price": price,
        #     "Category": cat,
        #     "Link": row['Link'],
        #     "GameCharImage": game_char_image
        # }
    finally:
        driver.quit()
    return result


count = 0
results = []
gamesTwo_files = set(os.listdir("gamesTwo"))
for file_name in gamesTwo_files:
    if file_name.endswith(".csv"):
        existing_file = f"skycoach_results_{file_name}.csv"
        existing_file_folder = os.listdir("game_services")
        if existing_file in existing_file_folder:
            print(f"Skipping {file_name} as results already exist.")
            continue
        df = pd.read_csv(f"gamesTwo/{file_name}")
        category = file_name.replace("_services.csv", "").replace("_", " ")
        if 'Link' in df.columns:
            with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers as needed
                futures = []
                for index, row in df.iterrows():
                    subcat = row.get("Subcategory", "")
                    if subcat:
                        print(f"Processing subcategory: {subcat}")
                    else:
                        print(f"Processing category: {category}")
                    link = row['Link']
                    if link and link.startswith("https://skycoach.gg/"):
                        print(f"Processing link: {link}")
                        # Use a temporary driver to get nested links
                        driver = webdriver.Edge(options=options)
                        driver.get(link)
                        page_source = driver.page_source
                        soup = BeautifulSoup(page_source, "html.parser")
                        container = soup.find("div", class_="card-list game-tag-page__container game-tag-page__products-list")
                        nestedlinks = []
                        if container:
                            li_tags = container.find_all("li")
                            for li in li_tags:
                                a_tag = li.find("a", href=True)
                                if a_tag:
                                    href = a_tag["href"]
                                    nestedlinks.append(href)
                        driver.quit()
                        print(nestedlinks)
                        for nestedlink in nestedlinks:
                            futures.append(executor.submit(scrape_service_options, nestedlink))
                for future in as_completed(futures):
                    result = future.result()
                    if result:
                        results.append(result)
                        count += 1
                        print(count)
        df_result = pd.DataFrame(results)
        df_result.to_csv(f"skycoach_results_{file_name}.csv", index=False)
        results = []
print("Done.")


Processing subcategory: nan
Processing link: https://skycoach.gg/diablo-4-boost/season


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


['/diablo-4-boost/gold-coins', '/diablo-4-boost/products/powerleveling-2099', '/diablo-4-boost/products/paragon-leveling-2204', '/diablo-4-boost/products/diablo-4-belial-kill-7956', '/diablo-4-boost/products/runewords-5937', '/diablo-4-boost/products/lair-boss-kill-7961', '/diablo-4-boost/products/lair-boss-key-packs-7963', '/diablo-4-boost/products/leveling-bundles-8012', '/diablo-4-boost/products/horadric-jewels-8253', '/diablo-4-boost/products/dante-pet-8254', '/diablo-4-boost/products/horadric-phials-8256', '/diablo-4-boost/products/horadric-spell-8258', '/diablo-4-boost/products/horadric-knowledge-rank-8264', '/diablo-4-boost/products/diablo-4-unique-gear-boost-2400', '/diablo-4-boost/products/lair-boss-keys-7962', '/diablo-4-boost/products/mythic-runewords-bundle-7994', '/diablo-4-boost/products/the-pit-of-artificers-4592', '/diablo-4-boost/products/torment-difficulties-5948', '/diablo-4-boost/products/boss-packs-bundle-6063', '/diablo-4-boost/products/mythic-unique-gear-5833']


NameError: name 'scrape_service_options' is not defined